In [ ]:
pip install flask numpy pandas sklearn tensorflow geopy joblib scikit-learn keyboard


In [18]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import joblib

# Load dataset
data = pd.read_csv("d.csv")  # Replace with your dataset path

# Preprocess data
messages = data["message"]
categories = data["category"]

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(categories)

# Transform text data to TF-IDF vectors
vectorizer = TfidfVectorizer(max_features=5000, stop_words="english")
X = vectorizer.fit_transform(messages)

# Train the model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X, encoded_labels)

# Save the trained components
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")
joblib.dump(model, "disaster_model.pkl")
joblib.dump(label_encoder, "label_encoder.pkl")

print("Model files saved: tfidf_vectorizer.pkl, disaster_model.pkl, label_encoder.pkl")


Model files saved: tfidf_vectorizer.pkl, disaster_model.pkl, label_encoder.pkl


In [19]:
import joblib
import csv
import logging
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import folium
from folium.plugins import MarkerCluster
import keyboard


In [20]:


logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
# Load pretrained models
try:
    vectorizer = joblib.load("tfidf_vectorizer.pkl")
    model = joblib.load("disaster_model.pkl")
    label_encoder = joblib.load("label_encoder.pkl")
except Exception as e:
    logging.error(f"Error loading models: {str(e)}")
    exit(1)

# Load NGO dataset
try:
    ngo_df = pd.read_csv("ngo_dataset.csv")
except Exception as e:
    logging.error(f"Error loading NGO dataset: {str(e)}")
    exit(1)

# Initialize geolocator
geolocator = Nominatim(user_agent="disaster_response_app")

In [21]:

def find_nearest_ngo(latitude, longitude):
    try:
        user_location = (latitude, longitude)
        ngo_data = ngo_df.copy()
        ngo_data["Distance"] = ngo_data.apply(
            lambda row: geodesic(user_location, (row["Latitude"], row["Longitude"])).kilometers, axis=1
        )
        nearest_ngo = ngo_data.loc[ngo_data["Distance"].idxmin()]
        return {
            "name": nearest_ngo["NGO Name"],
            "city": nearest_ngo["City"],
            "contact": nearest_ngo["Contact"],
            "email": nearest_ngo["Email"],
            "latitude": nearest_ngo["Latitude"],
            "longitude": nearest_ngo["Longitude"],
            "distance": nearest_ngo["Distance"]
        }
    except Exception as e:
        logging.error(f"NGO search error: {str(e)}")
        return None

In [22]:



def predict_message_local(message):
    try:
        # Preprocess and predict
        vectorized_message = vectorizer.transform([message])
        probabilities = model.predict_proba(vectorized_message)
        predicted_class = model.predict(vectorized_message)[0]
        confidence = max(probabilities[0])
        category_name = label_encoder.inverse_transform([predicted_class])[0]
        return category_name, confidence
    except Exception as e:
        logging.error(f"Prediction error: {str(e)}")
        return None, None

In [23]:

def get_geolocation(location_name):
    try:
        location = geolocator.geocode(location_name, timeout=10)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except Exception as e:
        logging.error(f"Geolocation error: {str(e)}")
        return None, None

In [24]:

def save_predictions_to_file(predictions, filename="predictions.csv"):
    try:
        with open(filename, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(["Message", "Category", "Confidence", "Latitude", "Longitude", "NGO Name", "NGO City", "NGO Contact", "NGO Email", "Distance (km)"])
            writer.writerows(predictions)
        logging.info(f"Predictions saved to {filename}")
    except Exception as e:
        logging.error(f"File saving error: {str(e)}")

In [25]:

def create_location_map(predictions, filename="locations_map.html"):
    try:
        disaster_map = folium.Map(location=[20.5937, 78.9629], zoom_start=5)  # Default: India
        marker_cluster = MarkerCluster().add_to(disaster_map)
        
        for prediction in predictions:
            message, category, confidence, latitude, longitude, ngo_name, ngo_city, ngo_contact, ngo_email, ngo_distance = prediction
            if latitude and longitude:
                popup = (
                    f"<b>Message:</b> {message}<br>"
                    f"<b>Category:</b> {category} (Confidence: {confidence:.2f})<br>"
                    f"<b>NGO:</b> {ngo_name} ({ngo_city})<br>"
                    f"<b>Contact:</b> {ngo_contact}, <b>Email:</b> {ngo_email}<br>"
                    f"<b>Distance:</b> {ngo_distance:.2f} km"
                )
                folium.Marker([latitude, longitude], popup=popup).add_to(marker_cluster)
        
        disaster_map.save(filename)
        logging.info(f"Map saved to {filename}")
    except Exception as e:
        logging.error(f"Map creation error: {str(e)}")

In [26]:
def cli_predictor():
    print("Welcome to the Enhanced Disaster Response Predictor!")
    print("Press 'Esc' at any time to exit.")
    predictions = []
    
    while True:
        # Check if Esc key is pressed
        if keyboard.is_pressed("esc"):
            print("\nExiting application. Goodbye!")
            break
        
        message = input("Enter a disaster message: ")
        if not message.strip():
            print("Message cannot be empty!")
            continue
        
        location = input("Enter a location (optional): ")
        category, confidence = predict_message_local(message)
        
        if location:
            latitude, longitude = get_geolocation(location)
            if latitude and longitude:
                nearest_ngo = find_nearest_ngo(latitude, longitude)
            else:
                nearest_ngo = None
        else:
            latitude, longitude = None, None
            nearest_ngo = None
        
        if category:
            print(f"Predicted category: {category} (Confidence: {confidence:.2f})")
            if latitude and longitude:
                print(f"Location coordinates: Latitude={latitude}, Longitude={longitude}")
                if nearest_ngo:
                    print(f"Nearest NGO: {nearest_ngo['name']} ({nearest_ngo['city']})")
                    print(f"Contact: {nearest_ngo['contact']}, Email: {nearest_ngo['email']}, Distance: {nearest_ngo['distance']:.2f} km")
                else:
                    print("No nearby NGO found.")
            predictions.append([
                message, category, confidence, latitude, longitude,
                nearest_ngo["name"] if nearest_ngo else None,
                nearest_ngo["city"] if nearest_ngo else None,
                nearest_ngo["contact"] if nearest_ngo else None,
                nearest_ngo["email"] if nearest_ngo else None,
                nearest_ngo["distance"] if nearest_ngo else None
            ])
        else:
            print("Could not predict category.")
        
        cont = input("Do you want to predict another message? (y/n/save/map): ").lower()
        if cont == "save":
            save_predictions_to_file(predictions)
        elif cont == "map":
            create_location_map(predictions)
        elif cont != "y":
            break

In [29]:

if __name__ == "__main__":
    cli_predictor()

Welcome to the Enhanced Disaster Response Predictor!
Press 'Esc' at any time to exit.
Predicted category: drought (Confidence: 0.57)
Location coordinates: Latitude=15.3004543, Longitude=74.0855134
Nearest NGO: Care & Share NGO (Bengaluru)
Contact: +91 9840907271, Email: ngo_552@help.org, Distance: 76.07 km
